#  ROCK-PAPER-SCISSORS

In [1]:
import cv2

top, bottom, right, left = 60, 350, 400, 625  #Rectangle positions
calibrationSetSize = 5   #Number of images are going to be taken to detect gestures. (Higher is more precise but slower.)
sleepTime = 0.1         #The time interval between images when taking photo
#(B, G, R)
rectangleColor = (255, 255, 255)
textColor = (255, 255, 255)
rectThickness = 2
recentPlayerMoves = []
fontPlain = cv2.FONT_HERSHEY_PLAIN
fontSimplex = cv2.FONT_HERSHEY_SIMPLEX

In [2]:
from tensorflow import keras
TRAINING_DIR ='Downloads/rps/'
training_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

train_generator = training_datagen.flow_from_directory(
        TRAINING_DIR,
        target_size=(150, 150),
        class_mode='categorical'
)

TESTING_DIR = 'Downloads/rps-test-set/'
testing_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

testing_generator = testing_datagen.flow_from_directory(
        TESTING_DIR,
        target_size=(150, 150),
        class_mode='categorical'
)

Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


In [3]:
model = keras.Sequential([
                        keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(150, 150, 3)),
                        keras.layers.MaxPooling2D(2,2),
                        keras.layers.Conv2D(128, (3,3), activation='relu'),
                        keras.layers.MaxPooling2D(2,2),
                        keras.layers.Conv2D(128, (3,3), activation='relu'),
                        keras.layers.MaxPooling2D(2,2),
                        keras.layers.Conv2D(128, (3,3), activation='relu'),
                        keras.layers.MaxPooling2D(2,2),
                        keras.layers.Flatten(),
                        keras.layers.Dropout(rate = 0.5),
                        keras.layers.Dense(512, activation='relu'),
                        keras.layers.Dense(3, activation='softmax')
                    ])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 128)     3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

In [4]:
import time
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
t1 = time.time()
history = model.fit_generator(train_generator, epochs = 0, validation_data = testing_generator, verbose = 1)
print(time.time() - t1)
model.save("rps.h5")

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 79 steps, validate for 12 steps
0.470095157623291


In [23]:
import matplotlib.pyplot as plt
import numpy as np
import random, os
path = r"random"
computerMoveImage = cv2.imread("/random/default.jpg")

(wx, wy, length) = (370, 120, 250)
(user_points, auto_points) = (0, 0)
(previous_prediction, prediction) = ("!", "@")
(predict, restart, auto) = ("", False, "")
classes = ["Paper", "Rock", "Scissor"]

video = cv2.VideoCapture(-1)

while(True):
    check, frame = video.read()

#frame = np.reshape(frame, (1, 150, 150, -1))
#gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#cv2.imshow('frame', gray)

    if restart:
        (user_points, auto_points) = (0, 0)
        restart = False
        
    if check:
        cv2.rectangle(frame, (left, top), (right, bottom), rectangleColor, rectThickness)
        computerMoveImage = random.choice([
            x for x in os.listdir(path)
            if os.path.isfile(os.path.join(path, x))
        ])
        print(computerMoveImage)
        roi_color = frame[150, 150]

        computerMoveImage = cv2.imread("./random/{}".format(computerMoveImage),-1)
        orig_mask = computerMoveImage[:,:,2]
 
        # Create the inverted mask for the mustache
        orig_mask_inv = cv2.bitwise_not(orig_mask)
 
        # Convert mustache image to BGR
        # and save the original image size (used later when re-sizing the image)
        computerMoveImage = computerMoveImage[:,:,0:3]
        origMustacheHeight, origMustacheWidth = computerMoveImage.shape[:2]
        cv2.rectangle(frame, (left, top), (right, bottom), rectangleColor, rectThickness)
        cv2.putText(frame, "Your Points : %d, Computer Points : %d"%(user_points, auto_points), (50, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1)
        cv2.putText(frame, "You : %s, Computer : %s"%(predict, auto), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1)
        roi = roi_color[100, 200]
 
        # roi_bg contains the original image only where the mustache is not
        # in the region that is the size of the mustache.
        roi_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)

        # roi_fg contains the image of the mustache only where the mustache is
        roi_fg = cv2.bitwise_and(mustache,mustache,mask = mask)

        # join the roi_bg and roi_fg
        dst = cv2.add(roi_bg,roi_fg)

        # place the joined image, saved to dst back over the original image
        roi_color[y1:y2, x1:x2] = dst
        frame = frame[wy : wy + length, wx : wx + length]
        frame = cv2.resize(frame, (150, 150))
        frame = np.expand_dims(frame, axis=0)
        prediction = model.predict(frame)[0]
        predict = classes[np.argmax(prediction)]
        #print(prediction)
        if predict != previous_prediction:
            auto = random.choice(classes)
            if predict == "Paper" and auto == "Rock":
                user_points += 1
            if predict == "Paper" and auto == "Scissor":
                auto_points += 1
            if predict == "Rock" and auto == "Paper":
                auto_points += 1
            if predict == "Rock" and auto == "Scissor":
                user_points += 1
            if predict == "Scissor" and auto == "Paper":
                user_points += 1
            if predict == "Scissor" and auto == "Rock":
                auto_points += 1
            previous_prediction = predict
        cv2.imshow("RockPaperScissors", frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    if key == ord('r'):
        restart = True
        
video.release()
cv2.destroyAllWindows()

rock.png


IndexError: too many indices for array

In [ ]:
import random, os, cv2
path = r"random"
picture = random.choice([
            x for x in os.listdir(path)
            if os.path.isfile(os.path.join(path, x))
        ])

print(picture)
computerMoveImage = cv2.imread("./random/{}".format(picture))
print(computerMoveImage)
